## Exploratory Data analysis & Feature engineering¶
What is done in this section
For each individual feature:

* explored nature of the feature and its relationship with the target variable (flight delay)
* treated null values
* treated outliers
* based on results from EDA, conducted feature transformations (e.g. binning/dummy coding)
* dropped variable if not informative of the target variable
* conducted feature selection with feature importance ranking technique



In [67]:
import pandas as pd
import plotly
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import cross_val_score
import timeit


In [68]:
df = pd.read_csv('../Data/Jan_2020_ontime.csv')

pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', -1)  # or 199

/Users/ej7793/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning:

Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.



In [69]:
df.describe()


,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEP_TIME,DEP_DEL15,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE,Unnamed: 21
count,607346.000000,607346.000000,607346.000000,607346.000000,607346.000000,6.073460e+05,607346.000000,6.073460e+05,600682.000000,600647.000000,600271.000000,599268.000000,607346.000000,607346.000000,607346.000000,0.0
mean,16.014354,3.955735,19994.137492,2622.365261,12657.389167,1.265743e+06,12657.196320,1.265724e+06,1331.512559,0.136326,1477.968924,0.137309,0.011407,0.001893,798.022341,NaN
std,8.990719,1.910205,375.754232,1822.545302,1524.407203,1.524405e+05,1524.279269,1.524277e+05,497.095168,0.343135,525.543294,0.344174,0.106193,0.043473,587.282639,NaN
min,1.000000,1.000000,19393.000000,1.000000,10135.000000,1.013506e+06,10135.000000,1.013506e+06,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,31.000000,NaN
25%,8.000000,2.000000,19790.000000,1070.000000,11292.000000,1.129202e+06,11292.000000,1.129202e+06,919.000000,0.000000,1058.000000,0.000000,0.000000,0.000000,369.000000,NaN
50%,16.000000,4.000000,19977.000000,2177.000000,12889.000000,1.288903e+06,12889.000000,1.288903e+06,1326.000000,0.000000,1513.000000,0.000000,0.000000,0.000000,641.000000,NaN
75%,24.000000,5.000000,20378.000000,4108.000000,14027.000000,1.402702e+06,14027.000000,1.402702e+06,1739.000000,0.000000,1918.000000,0.000000,0.000000,0.000000,1037.000000,NaN
max,31.000000,7.000000,20452.000000,6860.000000,16869.000000,1.686901e+06,16869.000000,1.686901e+06,2400.000000,1.000000,2400.000000,1.000000,1.000000,1.000000,5095.000000,NaN


In [70]:
df.isnull().any()

DAY_OF_MONTH             False
DAY_OF_WEEK              False
OP_UNIQUE_CARRIER        False
OP_CARRIER_AIRLINE_ID    False
OP_CARRIER               False
TAIL_NUM                 True 
OP_CARRIER_FL_NUM        False
ORIGIN_AIRPORT_ID        False
ORIGIN_AIRPORT_SEQ_ID    False
ORIGIN                   False
DEST_AIRPORT_ID          False
DEST_AIRPORT_SEQ_ID      False
DEST                     False
DEP_TIME                 True 
DEP_DEL15                True 
DEP_TIME_BLK             False
ARR_TIME                 True 
ARR_DEL15                True 
CANCELLED                False
DIVERTED                 False
DISTANCE                 False
Unnamed: 21              True 
dtype: bool

In [71]:
df.sample(10)

,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST,DEP_TIME,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE,Unnamed: 21
499715,26,7,AA,19805,AA,N406AN,2099,14107,1410702,PHX,14679,1467903,SAN,612.0,0.0,0600-0659,631.0,0.0,0.0,0.0,304.0,NaN
496474,26,7,UA,19977,UA,N829UA,2043,13930,1393007,ORD,12266,1226603,IAH,619.0,1.0,0600-0659,903.0,0.0,0.0,0.0,925.0,NaN
430040,23,4,B6,20409,B6,N570JB,2269,10721,1072102,BOS,11697,1169706,FLL,1207.0,0.0,1200-1259,1531.0,0.0,0.0,0.0,1237.0,NaN
362169,19,7,AA,19805,AA,N524UW,655,14100,1410005,PHL,14107,1410702,PHX,1521.0,0.0,1500-1559,1853.0,0.0,0.0,0.0,2075.0,NaN
128903,7,2,WN,19393,WN,N268WN,9,13204,1320402,MCO,11259,1125904,DAL,621.0,0.0,0600-0659,804.0,0.0,0.0,0.0,973.0,NaN
544581,28,2,DL,19790,DL,N703TW,757,12478,1247805,JFK,14771,1477104,SFO,658.0,0.0,0700-0759,1006.0,0.0,0.0,0.0,2586.0,NaN
413475,22,3,WN,19393,WN,N424WN,610,10693,1069302,BNA,11259,1125904,DAL,823.0,0.0,0800-0859,1012.0,0.0,0.0,0.0,623.0,NaN
349341,18,6,OO,20304,OO,N240SY,3592,14747,1474703,SEA,12441,1244102,JAC,1124.0,0.0,1100-1159,1406.0,0.0,0.0,0.0,621.0,NaN
384843,20,1,OH,20397,OH,N218PS,5652,11278,1127805,DCA,12884,1288403,LAN,1710.0,0.0,1700-1759,1844.0,0.0,0.0,0.0,479.0,NaN
253595,13,1,DL,19790,DL,N909DE,1915,10397,1039707,ATL,15016,1501606,STL,1750.0,0.0,1700-1759,1825.0,0.0,0.0,0.0,484.0,NaN


In [72]:
## 
##drop all not in the delayed category
df = df[df['DIVERTED'] == 0]
df = df[df['CANCELLED'] == 0]

df_filtered = df.drop(['DIVERTED', 'CANCELLED', 'Unnamed: 21'], axis = 1)
df_filtered.columns

Index(['DAY_OF_MONTH', 'DAY_OF_WEEK', 'OP_UNIQUE_CARRIER',
       'OP_CARRIER_AIRLINE_ID', 'OP_CARRIER', 'TAIL_NUM', 'OP_CARRIER_FL_NUM',
       'ORIGIN_AIRPORT_ID', 'ORIGIN_AIRPORT_SEQ_ID', 'ORIGIN',
       'DEST_AIRPORT_ID', 'DEST_AIRPORT_SEQ_ID', 'DEST', 'DEP_TIME',
       'DEP_DEL15', 'DEP_TIME_BLK', 'ARR_TIME', 'ARR_DEL15', 'DISTANCE'],
      dtype='object')

In [73]:
df = df_filtered

Day of month

In [74]:
df_day_of_month = pd.DataFrame(df.groupby(['DAY_OF_MONTH'])['ARR_DEL15'].value_counts(normalize = True))
df_day_of_month.rename({'ARR_DEL15': 'delayed_percentage'}, axis = 1, inplace = True)
df_day_of_month = df_day_of_month.reset_index()
df_day_of_month.head()

,DAY_OF_MONTH,ARR_DEL15,delayed_percentage
0,1,0.0,0.874784
1,1,1.0,0.125216
2,2,0.0,0.863082
3,2,1.0,0.136918
4,3,0.0,0.808468


There seems to be a weekly effect, delays spikes up every 7 days

In [75]:
import plotly.express as px
fig = px.line(df_day_of_month, x='DAY_OF_MONTH', y='delayed_percentage', color='ARR_DEL15')
fig.show()


Day of week

In [76]:
df_day_of_week = pd.DataFrame(df.groupby(['DAY_OF_WEEK'])['ARR_DEL15'].value_counts(normalize = True))
df_day_of_week.rename({'ARR_DEL15': 'delayed_percentage'}, axis = 1, inplace = True)
df_day_of_week = df_day_of_week.reset_index()
df_day_of_week

,DAY_OF_WEEK,ARR_DEL15,delayed_percentage
0,1,0.0,0.866037
1,1,1.0,0.133963
2,2,0.0,0.901865
3,2,1.0,0.098135
4,3,0.0,0.902344
5,3,1.0,0.097656
6,4,0.0,0.862480
7,4,1.0,0.137520
8,5,0.0,0.838231
9,5,1.0,0.161769


Saturday is the day with most delays, should transform the column to Saturday indicator

In [77]:
import plotly.express as px
fig = px.bar(df_day_of_week, x='DAY_OF_WEEK', y='delayed_percentage', color='ARR_DEL15')
fig.show()

In [78]:
df['Saturday'] = 0
df.loc[df['DAY_OF_WEEK'] ==6, 'Saturday'] = 1
df.Saturday.value_counts()

0    535734
1    63534 
Name: Saturday, dtype: int64

In [79]:
df['Tuesday_Wednesday'] = 0
df.loc[(df['DAY_OF_WEEK'] ==2) | (df['DAY_OF_WEEK'] ==3), 'Tuesday_Wednesday'] = 1
df.Tuesday_Wednesday.value_counts()

0    426943
1    172325
Name: Tuesday_Wednesday, dtype: int64

OP_UNIQUE_CARRIER transformation

In [80]:

df_OP_UNIQUE_CARRIER = pd.DataFrame(df.groupby(['OP_UNIQUE_CARRIER'])['ARR_DEL15'].value_counts(normalize = True))
df_OP_UNIQUE_CARRIER.rename({'ARR_DEL15': 'delayed_percentage'}, axis = 1, inplace = True)
df_OP_UNIQUE_CARRIER = df_OP_UNIQUE_CARRIER.reset_index()

In [81]:
import plotly.express as px
fig = px.bar(df_OP_UNIQUE_CARRIER, x='OP_UNIQUE_CARRIER', y='delayed_percentage', color='ARR_DEL15')
fig.show()

In [82]:
df_OP_UNIQUE_CARRIER = df_OP_UNIQUE_CARRIER[df_OP_UNIQUE_CARRIER['ARR_DEL15'] ==1.0]
df_OP_UNIQUE_CARRIER['CARRIER_rank'] = pd.qcut(df_OP_UNIQUE_CARRIER['delayed_percentage'], 17, labels = False)
#df_OP_UNIQUE_CARRIER = df_OP_UNIQUE_CARRIER[['OP_UNIQUE_CARRIER', 'CARRIER_rank']]
#df = pd.merge(df, df_OP_UNIQUE_CARRIER, how = 'left', on= 'OP_UNIQUE_CARRIER')



In [83]:
df_OP_UNIQUE_CARRIER = df_OP_UNIQUE_CARRIER[['OP_UNIQUE_CARRIER', 'CARRIER_rank']]
df = pd.merge(df, df_OP_UNIQUE_CARRIER, how = 'left', on= 'OP_UNIQUE_CARRIER')

In [84]:
df.columns

Index(['DAY_OF_MONTH', 'DAY_OF_WEEK', 'OP_UNIQUE_CARRIER',
       'OP_CARRIER_AIRLINE_ID', 'OP_CARRIER', 'TAIL_NUM', 'OP_CARRIER_FL_NUM',
       'ORIGIN_AIRPORT_ID', 'ORIGIN_AIRPORT_SEQ_ID', 'ORIGIN',
       'DEST_AIRPORT_ID', 'DEST_AIRPORT_SEQ_ID', 'DEST', 'DEP_TIME',
       'DEP_DEL15', 'DEP_TIME_BLK', 'ARR_TIME', 'ARR_DEL15', 'DISTANCE',
       'Saturday', 'Tuesday_Wednesday', 'CARRIER_rank'],
      dtype='object')

drop OP_CARRIER since it's the same as OP_UNIQUE_CARRIER, and drop OP_CARRIER_AIRLINE_ID since it is the unique identifier of each OP_UNIQUE_CARRIER

In [85]:
df.OP_CARRIER_AIRLINE_ID.nunique()

17

In [86]:
df['OP_UNIQUE_CARRIER'].equals(df['OP_CARRIER'])

True

tail_num is the legal name for each aircraft, drop this column since not informative of delays

In [87]:
df.TAIL_NUM.describe()

count     599268
unique    5443  
top       N488HA
freq      333   
Name: TAIL_NUM, dtype: object

In [88]:
df.OP_CARRIER_FL_NUM.nunique()

6719

In [89]:
df.ORIGIN_AIRPORT_SEQ_ID.nunique()

351

In [90]:
pd.DataFrame(df.groupby(['ORIGIN'])['ARR_DEL15'].value_counts(normalize = True))

ARR_DEL15
ORIGIN ARR_DEL15           
ABE    0.0        0.832869 
       1.0        0.167131 
ABI    0.0        0.717791 
       1.0        0.282209 
ABQ    0.0        0.900274 
       1.0        0.099726 
ABR    0.0        0.783333 
       1.0        0.216667 
ABY    0.0        0.940476 
       1.0        0.059524 
ACT    0.0        0.777778 
       1.0        0.222222 
ACV    0.0        0.827027 
       1.0        0.172973 
ACY    0.0        0.923077 
       1.0        0.076923 
ADK    0.0        0.555556 
       1.0        0.444444 
ADQ    0.0        0.680000 
       1.0        0.320000 
AEX    0.0        0.839216 
       1.0        0.160784 
AGS    0.0        0.830137 
       1.0        0.169863 
ALB    0.0        0.866221 
       1.0        0.133779 
ALO    0.0        0.758621 
       1.0        0.241379 
AMA    0.0        0.864450 
       1.0        0.135550 
ANC    0.0        0.858340 
       1.0        0.141660 
APN    0.0        0.807692 
       1.0        0.192308 
ASE    0.0        0.694205 
       1.0        0.305795 
ATL    0.0        0.898264 
       1.0        0.101736 
ATW    0.0        0.816568 
       1.0        0.183432 
ATY    0.0        0.872727 
       1.0        0.127273 
AUS    0.0        0.903238 
       1.0        0.096762 
AVL    0.0        0.809461 
       1.0        0.190539 
AVP    0.0        0.802867 
       1.0        0.197133 
AZA    0.0        0.755647 
       1.0        0.244353 
AZO    0.0        0.846473 
       1.0        0.153527 
BDL    0.0        0.902686 
       1.0        0.097314 
BET    0.0        0.779412 
       1.0        0.220588 
BFF    0.0        0.846154 
       1.0        0.153846 
BFL    0.0        0.845833 
       1.0        0.154167 
BFM    0.0        0.923077 
       1.0        0.076923 
BGM    0.0        0.883333 
       1.0        0.116667 
BGR    0.0        0.788820 
       1.0        0.211180 
BHM    0.0        0.872299 
       1.0        0.127701 
BIL    0.0        0.912338 
       1.0        0.087662 
BIS    0.0        0.811209 
       1.0        0.188791 
BJI    0.0        0.852459 
       1.0        0.147541 
BLI    0.0        0.878788 
       1.0        0.121212 
BLV    0.0        0.738462 
       1.0        0.261538 
BMI    0.0        0.818533 
       1.0        0.181467 
BNA    0.0        0.896231 
       1.0        0.103769 
BOI    0.0        0.872276 
       1.0        0.127724 
BOS    0.0        0.886888 
       1.0        0.113112 
BPT    0.0        0.830986 
       1.0        0.169014 
BQK    0.0        0.930233 
       1.0        0.069767 
BQN    0.0        0.746114 
       1.0        0.253886 
BRD    0.0        0.846154 
       1.0        0.153846 
BRO    0.0        0.917127 
       1.0        0.082873 
BRW    0.0        0.870968 
       1.0        0.129032 
BTM    0.0        0.965517 
       1.0        0.034483 
BTR    0.0        0.839255 
       1.0        0.160745 
BTV    0.0        0.818182 
       1.0        0.181818 
BUF    0.0        0.879821 
       1.0        0.120179 
BUR    0.0        0.904762 
       1.0        0.095238 
BWI    0.0        0.880767 
       1.0        0.119233 
BZN    0.0        0.863937 
       1.0        0.136063 
CAE    0.0        0.840510 
       1.0        0.159490 
CAK    0.0        0.807933 
       1.0        0.192067 
CDC    0.0        0.962963 
       1.0        0.037037 
CDV    0.0        0.750000 
       1.0        0.250000 
CGI    0.0        0.833333 
       1.0        0.166667 
CHA    0.0        0.875994 
       1.0        0.124006 
CHO    0.0        0.820183 
       1.0        0.179817 
CHS    0.0        0.872802 
       1.0        0.127198 
CID    0.0        0.752368 
       1.0        0.247632 
CIU    0.0        0.854545 
       1.0        0.145455 
CKB    0.0        0.811594 
       1.0        0.188406 
CLE    0.0        0.886723 
       1.0        0.113277 
CLL    0.0        0.910256 
       1.0        0.089744 
CLT    0.0        0.819468 
       1.0        0.180532 
CMH    0.0        0.890993 
       1.0    

In [91]:
df_ORIGIN = pd.DataFrame(df.groupby(['ORIGIN'])['ARR_DEL15'].value_counts(normalize = True))
df_ORIGIN.rename({'ARR_DEL15': 'delayed_percentage'}, axis = 1, inplace = True)
df_ORIGIN = df_ORIGIN.reset_index()

In [92]:
df_ORIGIN = df_ORIGIN[df_ORIGIN['ARR_DEL15'] ==1.0]
df_ORIGIN['ORIGIN_rank'] = pd.qcut(df_ORIGIN['delayed_percentage'], 10, labels = False)
df_ORIGIN = df_ORIGIN[['ORIGIN', 'ORIGIN_rank']]
df = pd.merge(df, df_ORIGIN, how = 'left', on= 'ORIGIN')

In [93]:
len(df[df.ORIGIN_rank.isnull()])

11

In [94]:
## removing rows with null ORIGIN_rank 
df = df.loc[df['ORIGIN_rank'].notnull()]
len(df)

599257

In [95]:
df.columns

Index(['DAY_OF_MONTH', 'DAY_OF_WEEK', 'OP_UNIQUE_CARRIER',
       'OP_CARRIER_AIRLINE_ID', 'OP_CARRIER', 'TAIL_NUM', 'OP_CARRIER_FL_NUM',
       'ORIGIN_AIRPORT_ID', 'ORIGIN_AIRPORT_SEQ_ID', 'ORIGIN',
       'DEST_AIRPORT_ID', 'DEST_AIRPORT_SEQ_ID', 'DEST', 'DEP_TIME',
       'DEP_DEL15', 'DEP_TIME_BLK', 'ARR_TIME', 'ARR_DEL15', 'DISTANCE',
       'Saturday', 'Tuesday_Wednesday', 'CARRIER_rank', 'ORIGIN_rank'],
      dtype='object')

In [96]:
df_DEST = pd.DataFrame(df.groupby(['DEST'])['ARR_DEL15'].value_counts(normalize = True))
df_DEST.rename({'ARR_DEL15': 'delayed_percentage'}, axis = 1, inplace = True)
df_DEST = df_DEST.reset_index()
df_DEST = df_DEST[df_DEST['ARR_DEL15'] ==1.0]
df_DEST['DEST_rank'] = pd.qcut(df_DEST['delayed_percentage'], 10, labels = False)
df_DEST = df_DEST[['DEST', 'DEST_rank']]
df = pd.merge(df, df_DEST, how = 'left', on= 'DEST')

In [97]:
len(df[df['DEST_rank'].isnull()])

22

In [98]:
## removing rows with null DEST_rank
df = df[df['DEST_rank'].notnull()]
len(df)

599235

In [99]:
df.columns

Index(['DAY_OF_MONTH', 'DAY_OF_WEEK', 'OP_UNIQUE_CARRIER',
       'OP_CARRIER_AIRLINE_ID', 'OP_CARRIER', 'TAIL_NUM', 'OP_CARRIER_FL_NUM',
       'ORIGIN_AIRPORT_ID', 'ORIGIN_AIRPORT_SEQ_ID', 'ORIGIN',
       'DEST_AIRPORT_ID', 'DEST_AIRPORT_SEQ_ID', 'DEST', 'DEP_TIME',
       'DEP_DEL15', 'DEP_TIME_BLK', 'ARR_TIME', 'ARR_DEL15', 'DISTANCE',
       'Saturday', 'Tuesday_Wednesday', 'CARRIER_rank', 'ORIGIN_rank',
       'DEST_rank'],
      dtype='object')

In [100]:
#df_filtered = df.drop(['OP_UNIQUE_CARRIER', 'OP_CARRIER_AIRLINE_ID', 'OP_CARRIER', 'TAIL_NUM', 'OP_CARRIER_FL_NUM'], axis = 1)

In [101]:
import math
df['DEP_HOUR'] = (df['DEP_TIME']/100)

df['DEP_HOUR'] = df['DEP_HOUR'].apply(lambda x: math.floor(x))
df.DEP_HOUR.describe()

count    599235.000000
mean     13.009170    
std      4.981942     
min      0.000000     
25%      9.000000     
50%      13.000000    
75%      17.000000    
max      24.000000    
Name: DEP_HOUR, dtype: float64

In [102]:
df_DEP_HOUR = pd.DataFrame(df.groupby(['DEP_HOUR'])['ARR_DEL15'].value_counts(normalize = True))
df_DEP_HOUR.rename({'ARR_DEL15': 'delayed_percentage'}, axis = 1, inplace = True)
df_DEP_HOUR = df_DEP_HOUR.reset_index().sort_values(by = ['delayed_percentage'])
df_DEP_HOUR = df_DEP_HOUR[df_DEP_HOUR['ARR_DEL15'] == 1.0]

df_DEP_HOUR['DEP_HOUR_rank'] = pd.qcut(df_DEP_HOUR['delayed_percentage'], 10, labels = False)
df_DEP_HOUR = df_DEP_HOUR[['DEP_HOUR', 'DEP_HOUR_rank']]
df = pd.merge(df, df_DEP_HOUR, how = 'left', on= 'DEP_HOUR')

**Hours between 22 PM and 3 AM has the delay rates larger than 20%** 
Create a binalry feature for the DEP_HOUR

In [103]:
df['LATE_NIGHT'] = 0
df.loc[(df['DEP_HOUR']>= 22)|(df['DEP_HOUR']<= 3), 'LATE_NIGHT'] =1
df['LATE_NIGHT'].describe()

count    599235.000000
mean     0.038092     
std      0.191418     
min      0.000000     
25%      0.000000     
50%      0.000000     
75%      0.000000     
max      1.000000     
Name: LATE_NIGHT, dtype: float64

**DEP_DEL15 & ARR_DEL15 are highly correlated**

In [104]:
df[['DEP_DEL15','ARR_DEL15']].corr()

,DEP_DEL15,ARR_DEL15
DEP_DEL15,1.00000,0.71126
ARR_DEL15,0.71126,1.00000


In [106]:
df.columns 

Index(['DAY_OF_MONTH', 'DAY_OF_WEEK', 'OP_UNIQUE_CARRIER',
       'OP_CARRIER_AIRLINE_ID', 'OP_CARRIER', 'TAIL_NUM', 'OP_CARRIER_FL_NUM',
       'ORIGIN_AIRPORT_ID', 'ORIGIN_AIRPORT_SEQ_ID', 'ORIGIN',
       'DEST_AIRPORT_ID', 'DEST_AIRPORT_SEQ_ID', 'DEST', 'DEP_TIME',
       'DEP_DEL15', 'DEP_TIME_BLK', 'ARR_TIME', 'ARR_DEL15', 'DISTANCE',
       'Saturday', 'Tuesday_Wednesday', 'CARRIER_rank', 'ORIGIN_rank',
       'DEST_rank', 'DEP_HOUR', 'DEP_HOUR_rank', 'LATE_NIGHT'],
      dtype='object')

In [107]:
df['DISTANCE_rank'] = pd.qcut(df['DISTANCE'], 25, labels = False)
df.groupby(['DISTANCE_rank'])['ARR_DEL15'].value_counts(normalize =True)

DISTANCE_rank  ARR_DEL15
0              0.0          0.856769
               1.0          0.143231
1              0.0          0.849778
               1.0          0.150222
2              0.0          0.862325
               1.0          0.137675
3              0.0          0.863531
               1.0          0.136469
4              0.0          0.874451
               1.0          0.125549
5              0.0          0.875232
               1.0          0.124768
6              0.0          0.878007
               1.0          0.121993
7              0.0          0.872038
               1.0          0.127962
8              0.0          0.871759
               1.0          0.128241
9              0.0          0.869892
               1.0          0.130108
10             0.0          0.867545
               1.0          0.132455
11             0.0          0.874947
               1.0          0.125053
12             0.0          0.858023
               1.0          0.141977
13           

There doesn't seem to be a relationship between distance and delays, drop this variable 

**Final Feature list**

In [108]:
df_selected = df[['Saturday', 'Tuesday_Wednesday', 'CARRIER_rank', 'ORIGIN_rank',
       'DEST_rank','LATE_NIGHT', 'DEP_HOUR_rank',  'DEP_DEL15', 'ARR_DEL15']]

In [109]:
df_selected.isnull().any()

Saturday             False
Tuesday_Wednesday    False
CARRIER_rank         False
ORIGIN_rank          False
DEST_rank            False
LATE_NIGHT           False
DEP_HOUR_rank        False
DEP_DEL15            False
ARR_DEL15            False
dtype: bool

In [111]:
df_selected.to_csv('../Data/feature_engineered.csv', index = False)